In [1]:
import os
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='element-deeplabcut', ("Please move to the "
                                                              + "element directory")

In [2]:
import datajoint as dj
from pathlib import Path
import yaml

In [3]:
dj.conn()

[2023-10-12 16:45:35,059][INFO]: Connecting root@fakeservices.datajoint.io:3306
[2023-10-12 16:45:35,066][INFO]: Connected root@fakeservices.datajoint.io:3306


DataJoint connection (connected) root@fakeservices.datajoint.io:3306

In [4]:
from tutorial_pipeline import lab, subject, session, train, model  

[2023-10-12 16:45:36,295][WARNING]: lab.Project and related tables will be removed in a future version of Element Lab. Please use the project schema.


In [5]:
dj.list_schemas()

['mysql',
 'performance_schema',
 'sys',
 'neuro_lab',
 'neuro_subject',
 'neuro_session',
 'neuro_train',
 'neuro_model']

In [ ]:
dj.config

In [ ]:
(
    dj.Diagram(subject) 
    + dj.Diagram(lab) 
    + dj.Diagram(session) 
    + dj.Diagram(model) 
    + dj.Diagram(train)
)

In [ ]:
dj.Diagram(model) + dj.Diagram(train)

## Step 2 - Insert Subject, Session, and Behavior Videos

In [11]:
subject.Subject()

subject,subject_nickname,sex,subject_birth_date,subject_description


In [12]:
# Subject and Session tables
subject.Subject.insert1(
    dict(
        subject="subject6",
        sex="F",
        subject_birth_date="2020-01-01",
        subject_description="hneih_E105",
    ),
    skip_duplicates=True,
)

In [13]:
#Definition of the dictionary named "session_keys"
session_keys = [
    dict(subject="subject6", session_datetime="2021-06-02 14:04:22"),
    dict(subject="subject6", session_datetime="2021-06-03 14:43:10"),
]

#Insert this dictionary in the Session table
session.Session.insert(session_keys, skip_duplicates=True)
session.Session()

subject,session_datetime
subject6,2021-06-02 14:04:22
subject6,2021-06-03 14:43:10


In [22]:
### VideoRecording
recording_key = {'subject': 'subject6',
       'session_datetime': '2021-06-02 14:04:22',
       'recording_id': '1'}
model.VideoRecording.insert1({**recording_key, 'device': 'Camera1'}, skip_duplicates=True)

In [20]:
### VideoRecording.File

video_files = ["./example_data/inbox/from_top_tracking-DataJoint-2023-10-11/videos/test.mp4"]

model.VideoRecording.File.insert({
    **recording_key, 
    'file_id': v_idx, 
    'file_path': Path(f)} for v_idx, f in enumerate(video_files))

In [21]:
### RecordingInfo
model.RecordingInfo.populate()
model.RecordingInfo()

subject,session_datetime,recording_id,px_height height in pixels,px_width width in pixels,nframes number of frames,fps (Hz) frames per second,recording_datetime Datetime for the start of the recording,recording_duration video duration (s) from nframes / fps
subject6,2021-06-02 14:04:22,1,500,500,18000,60,None,300.0


## Step 3 - DLC inference task

In [23]:
recording_key

{'subject': 'subject6',
 'session_datetime': '2021-06-02 14:04:22',
 'recording_id': '1'}

In [35]:
task_key = {**recording_key, 'model_name': 'from_top_tracking_model_test'}

In [27]:
model.PoseEstimationTask.insert1(
    {**task_key,
     'task_mode': 'load',
     'pose_estimation_output_dir': './example_data/outbox/from_top_tracking-DataJoint-2023-10-11/videos/device_1_recording_1_model_from_top_tracking_100000_maxiters'
     })

In [31]:
### PoseEstimation
model.PoseEstimation.populate()

In [32]:
### Results
model.PoseEstimation.BodyPartPosition()

subject,session_datetime,recording_id,model_name User-friendly model name,body_part,frame_index frame index in model,x_pos,y_pos,z_pos,likelihood
subject6,2021-06-02 14:04:22,1,from_top_tracking_model_test,head,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
subject6,2021-06-02 14:04:22,1,from_top_tracking_model_test,tailbase,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=


In [46]:
df = (model.PoseEstimation.BodyPartPosition & task_key).fetch(format='frame').reset_index()

In [47]:
df

,subject,session_datetime,recording_id,model_name,body_part,frame_index,x_pos,y_pos,z_pos,likelihood
0,subject6,2021-06-02 14:04:22,1,from_top_tracking_model_test,head,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[273.9966125488281, 274.1033630371094, 274.032...","[314.97100830078125, 315.1459655761719, 315.13...",None,"[0.999998927116394, 0.999998927116394, 0.99999..."
1,subject6,2021-06-02 14:04:22,1,from_top_tracking_model_test,tailbase,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[254.29002380371094, 254.2755584716797, 254.26...","[275.48602294921875, 275.44000244140625, 275.4...",None,"[0.9999996423721313, 0.9999996423721313, 0.999..."


In [48]:
df = df.explode(['frame_index', 'x_pos', 'y_pos', 'likelihood']).reset_index()
df

,index,subject,session_datetime,recording_id,model_name,body_part,frame_index,x_pos,y_pos,z_pos,likelihood
0,0,subject6,2021-06-02 14:04:22,1,from_top_tracking_model_test,head,0,273.996613,314.971008,None,0.999999
1,0,subject6,2021-06-02 14:04:22,1,from_top_tracking_model_test,head,1,274.103363,315.145966,None,0.999999
2,0,subject6,2021-06-02 14:04:22,1,from_top_tracking_model_test,head,2,274.032654,315.133331,None,0.999999
3,0,subject6,2021-06-02 14:04:22,1,from_top_tracking_model_test,head,3,274.025238,315.152283,None,0.999999
4,0,subject6,2021-06-02 14:04:22,1,from_top_tracking_model_test,head,4,274.073181,315.173248,None,0.999999
...,...,...,...,...,...,...,...,...,...,...,...
119995,1,subject6,2021-06-02 14:04:22,1,from_top_tracking_model_test,tailbase,59995,323.293884,33.214066,None,1.0
119996,1,subject6,2021-06-02 14:04:22,1,from_top_tracking_model_test,tailbase,59996,321.602264,32.794708,None,1.0
119997,1,subject6,2021-06-02 14:04:22,1,from_top_tracking_model_test,tailbase,59997,320.173981,32.857304,None,1.0
119998,1,subject6,2021-06-02 14:04:22,1,from_top_tracking_model_test,tailbase,59998,318.708618,33.147358,None,0.999999


## Step 1 - Register an existing model in DataJoint pipeline

A DeepLabCut model is defined in a DLC-specific folder structure with a file named `config.yaml` that contains the specifications of a DLC model.

To "register" this DLC model with DataJoint, you can just specify this config file. See example below

In [33]:
config_file_rel = "./example_data/inbox/from_top_tracking-DataJoint-2023-10-11/config.yaml"

In [7]:
model.Model.insert_new_model(model_name='from_top_tracking_model_test',
                             dlc_config=config_file_rel,
                             shuffle=1,
                             trainingsetindex=0,
                             model_description='Model in example data: from_top_tracking model')

2023-10-12 16:45:52.864682: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-12 16:45:52.964849: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.9/site-packages/cv2/../../lib64:
2023-10-12 16:45:52.964877: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-10-12 16:45:52.985085: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been regis

Loading DLC 2.3.7...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)
--- DLC Model specification to be inserted ---
	model_name: from_top_tracking_model_test
	model_description: Model in example data: from_top_tracking model
	scorer: DLCresnet50fromtoptrackingOct11shuffle1
	task: from_top_tracking
	date: Oct11
	iteration: 0
	snapshotindex: -1
	shuffle: 1
	trainingsetindex: 0
	project_path: inbox/from_top_tracking-DataJoint-2023-10-11
	paramset_idx: None
	-- Template/Contents of config.yaml --
		Task: from_top_tracking
		scorer: DataJoint
		date: Oct11
		multianimalproject: False
		identity: None
		project_path: /workspaces/element-deeplabcut/example_data/inbox/from_top_tracking-DataJoint-2023-10-11
		video_sets: {'/Users/milagros/Desktop/from_top_tracking-DataJoint-2023-10-11/videos/test.mp4': {'crop': '0, 500, 0, 500'}, '/Users/milagros/Desktop/from_top_tracking-DataJoint-2023-10-11/videos/train1.mp4': {'crop': '0, 500, 0, 500'}}
		bodyparts: